<a href="https://colab.research.google.com/github/JohnVorwald/docs/blob/master/cec_trim_loads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Cell 1: Setup
import numpy as np
import csv, os, time, shutil
from dataclasses import dataclass

# Conversions
FT2M = 0.3048
KT2MPS = 0.514444
LBF2N = 4.4482216152605

# Parameters
@dataclass
class MRParams:
    rho: float = 1.225
    Nb: int = 2
    R: float = 125.0 * FT2M
    Rhub: float = 0.06 * (125.0 * FT2M)
    Nelem: int = 28
    c_root: float = 10.0 * FT2M
    c_tip: float  = 10.0 * FT2M
    Omega: float = 6.0
    Vax: float = 0.0
    Vx: float = 0.0
    Vy: float = 0.0
    alpha0: float = np.deg2rad(-2.0)
    Cla: float = 2*np.pi
    CD0: float = 0.012
    kCD: float = 0.02
    theta_collect_deg: float = 10.0
    twist_deg: float = -8.0
    elevon_start_frac: float = 0.80
    Ke_elevon: float = 1.0
    elevon_delta1c_deg: float = 0.0
    elevon_delta1s_deg: float = 0.0
    use_prandtl: bool = True
    max_iter: int = 80
    tol: float = 6e-4

@dataclass
class PropParams:
    # Two props per MR blade (4 total) at these radii
    r_in: float = 60.0 * FT2M
    r_out: float = 95.0 * FT2M
    # Prop geometry
    Rp: float = 12.5 * FT2M
    Bp: int = 4
    Nelem: int = 18
    sigma_p: float = 0.10
    theta_p_in_deg: float = 16.0
    theta_p_out_deg: float = 16.0
    omega_in_nom: float = 850.0 * 2*np.pi/60.0
    omega_out_nom: float = 900.0 * 2*np.pi/60.0
    # Flapping per prop blade (small-angle steady surrogate)
    Ibeta: float = 0.12
    kbeta: float = (2*np.pi*5.0)**2 * 0.12
    e_beta: float = 0.03
    m_eff: float = 2.0
    # Aero
    alpha0: float = np.deg2rad(-2.0)
    Cla: float = 2*np.pi
    CD0: float = 0.012
    kCD: float = 0.02

@dataclass
class SectionStruct:
    t_web: float = 0.010
    cap_area: float = 2e-4
    cap_sep: float = 0.25
    E: float = 70e9
    G: float = 27e9
    rho_mat: float = 1600.0

# Utility: make output dir
def make_dir(path):
    os.makedirs(path, exist_ok=True)
    return path

In [ ]:
# Cell 2: Helpers

def spanwise_geometry_linear(Rhub, R, Nelem, c_root, c_tip, theta_collect_deg, twist_deg):
    r = np.linspace(Rhub, R, Nelem)
    rr = r/R
    rr_lin = (rr - rr[0])/(rr[-1]-rr[0] + 1e-12)
    chord = c_root + (c_tip - c_root) * rr_lin
    theta_geom = np.deg2rad(theta_collect_deg + twist_deg * rr_lin)
    dr = np.gradient(r)
    return r, dr, chord, theta_geom

def prandtl_F(Nb, r, R, Rhub, phi):
    s = abs(np.sin(phi))
    if s < 1e-8:
        return 1.0
    f_tip = (Nb/2.0)*(R-r)/(r*s)
    f_hub = (Nb/2.0)*(r-Rhub)/(r*s)
    F = (2/np.pi)*np.arccos(np.exp(-f_tip))*(2/np.pi)*np.arccos(np.exp(-f_hub))
    return float(np.clip(F, 1e-3, 1.0))

def elevon_delta_theta(p: MRParams, r, psi):
    if r < p.elevon_start_frac * p.R:
        return 0.0
    d1c = np.deg2rad(p.elevon_delta1c_deg)
    d1s = np.deg2rad(p.elevon_delta1s_deg)
    return p.Ke_elevon * (d1c*np.cos(psi) + d1s*np.sin(psi))

def section_properties(chord, sec: SectionStruct):
    sep = sec.cap_sep * chord
    A_caps = 2*sec.cap_area
    Ixx_caps = sec.cap_area * (sep/2)**2 * 2.0
    Iyy_caps = (2/12)*sec.cap_area*(0.15*chord)**2
    perimeter = 2.0*(chord + 0.15*chord)
    area_cell = chord * 0.12*chord
    J = 4.0*(area_cell**2) / (perimeter / max(sec.t_web,1e-6))
    Ixx = Ixx_caps
    Iyy = Iyy_caps
    return Ixx, Iyy, J

In [ ]:

# Stabilization patch: robust aero, BEMT, and trim Newton

import numpy as np
from dataclasses import dataclass

# 1) Safe math guards
def safe_clip(x, lo, hi):
    return np.minimum(np.maximum(x, lo), hi)

def safe_sq(x, cap=600.0):
    # bound magnitude before squaring to prevent overflow
    return safe_clip(x, -cap, cap)**2

def finite_or(x, fallback):
    if np.any(~np.isfinite(x)):
        return fallback
    return x

# 2) Aerodynamics: linear lift with stall clipping; quadratic drag; positive floor on Cd
@dataclass
class AeroModel:
    Cla: float = 2*np.pi
    alpha0: float = np.deg2rad(-2.0)
    CD0: float = 0.012
    kCD: float = 0.02
    alpha_lin_deg: float = 12.0   # linear range
    cd_floor: float = 1.5e-3

AERO_MR = AeroModel()
AERO_PROP = AeroModel()

def section_lift_drag(alpha, chord, q, aero: AeroModel):
    # clip alpha to avoid unrealistic Cl growth
    alpha_clip = safe_clip(alpha, np.deg2rad(-aero.alpha_lin_deg), np.deg2rad(aero.alpha_lin_deg))
    Cl = aero.Cla * alpha_clip
    Cd = aero.CD0 + aero.kCD * Cl*Cl
    Cd = np.maximum(Cd, aero.cd_floor)
    L  = q * chord * Cl
    D  = q * chord * Cd
    return L, D, Cl, Cd

# 3) Prandtl guard
def prandtl_F_guarded(Nb, r, R, Rhub, phi, Fmin=0.05):
    s = abs(np.sin(phi))
    if s < 1e-8:
        return 1.0
    f_tip = (Nb/2.0)*(R-r)/(max(r, 1e-6)*s)
    f_hub = (Nb/2.0)*(r-Rhub)/(max(r, 1e-6)*s)
    F = (2/np.pi)*np.arccos(np.exp(-safe_clip(f_tip, 0.0, 50.0))) * (2/np.pi)*np.arccos(np.exp(-safe_clip(f_hub, 0.0, 50.0)))
    return float(np.clip(F, Fmin, 1.0))

# 4) Replace your mr_bemt_azimuth with this robust version (same signature)
def mr_bemt_azimuth(p, sec, az_count=24, log_every=5):
    r, dr, chord, theta_geom = spanwise_geometry_linear(p.Rhub, p.R, p.Nelem, p.c_root, p.c_tip,
                                                        p.theta_collect_deg, p.twist_deg)
    a  = np.full_like(r, 0.05)   # axial induction (scaled to ΩR in this formulation)
    ap = np.full_like(r, 0.00)   # swirl induction
    psi = np.linspace(0, 2*np.pi, az_count, endpoint=False)
    iter_logs = []
    relax_a  = 0.35
    relax_ap = 0.35

    # arrays to carry through the last finite values in case of NaN
    dT_r = np.zeros_like(r); dQ_r = np.zeros_like(r)
    Nn = np.zeros_like(r); Nt = np.zeros_like(r)

    for it in range(1, p.max_iter+1):
        a_old = a.copy(); ap_old = ap.copy()
        dT_r.fill(0.0); dQ_r.fill(0.0)
        Nn.fill(0.0); Nt.fill(0.0)

        for i, ri in enumerate(r):
            ci = chord[i]
            UT_rot = p.Omega*ri
            dT_acc = 0.0; dQ_acc = 0.0
            Nn_i = 0.0; Nt_i = 0.0

            for ps in psi:
                theta_loc = theta_geom[i] + elevon_delta_theta(p, ri, ps)
                # edgewise inflow
                Vt_ext = -p.Vx*np.sin(ps) + p.Vy*np.cos(ps)
                # clamp local velocities before squaring
                Ut = safe_clip(UT_rot*(1.0 + ap[i]) + Vt_ext, -300.0, 300.0)
                Un = safe_clip(p.Vax + a[i]*p.Omega*p.R, -150.0, 150.0)

                phi = np.arctan2(Un, Ut)
                alpha = theta_loc - phi - AERO_MR.alpha0
                q  = 0.5*p.rho*(safe_sq(Ut, 400.0) + safe_sq(Un, 200.0))

                F  = prandtl_F_guarded(p.Nb, ri, p.R, p.Rhub, phi) if p.use_prandtl else 1.0
                L, D, _, _ = section_lift_drag(alpha, ci, q, AERO_MR)

                dT_acc += (L*np.cos(phi) - D*np.sin(phi)) * dr[i] * F
                dQ_acc += (L*np.sin(phi) + D*np.cos(phi)) * ri * dr[i] * F
                Nn_i   += (L*np.cos(phi) - D*np.sin(phi))
                Nt_i   += (L*np.sin(phi) + D*np.cos(phi))

            # azimuth-average
            dT_r[i] = dT_acc/az_count
            dQ_r[i] = dQ_acc/az_count
            Nn[i]   = Nn_i/az_count
            Nt[i]   = Nt_i/az_count

            # momentum update with robust denominators
            A_ring = 2*np.pi*max(ri, 1e-6)*max(dr[i], 1e-6)
            denom_a  = 2*p.rho*A_ring*max(p.Vax + a[i]*p.Omega*p.R, 1e-3)*p.Omega*p.R
            denom_ap = 2*p.rho*A_ring*(p.Omega*ri)**2*max(ri,1e-6)

            a_new  = (dT_r[i]/(denom_a + 1e-8))
            ap_new = (dQ_r[i]/(denom_ap + 1e-8))
            # relax and clamp to keep in physical range
            a[i]  = safe_clip((1-relax_a)*a[i]  + relax_a*a_new,  -0.4, 0.6)
            ap[i] = safe_clip((1-relax_ap)*ap[i] + relax_ap*ap_new, -0.9, 0.9)

        # if NaNs sneak in, roll back to previous step and tighten relaxation
        if not np.isfinite(a).all() or not np.isfinite(ap).all():
            a = finite_or(a_old, np.zeros_like(a_old))
            ap = finite_or(ap_old, np.zeros_like(ap_old))
            relax_a  *= 0.5
            relax_ap *= 0.5

        err_a = float(np.linalg.norm(a-a_old, np.inf))
        err_ap= float(np.linalg.norm(ap-ap_old, np.inf))
        Fz_est = 2.0*float(np.sum(dT_r))
        Q_est  = 2.0*float(np.sum(dQ_r))
        if (it % log_every == 0) or (it == p.max_iter) or (max(err_a, err_ap) < p.tol):
            iter_logs.append((it, err_a, err_ap, Fz_est, Q_est))
        if max(err_a, err_ap) < p.tol:
            break

    # hub totals and azimuthal distribution (use current a, ap but guard velocities)
    Fx=Fy=Fz=Mx=My=Q=0.0
    hub_az = []
    for ps in psi:
        Fx_ps=Fy_ps=Fz_ps=Mx_ps=My_ps=Q_ps=0.0
        for i, ri in enumerate(r):
            ci = chord[i]
            theta_loc = theta_geom[i] + elevon_delta_theta(p, ri, ps)
            Vt_ext = -p.Vx*np.sin(ps) + p.Vy*np.cos(ps)
            Ut = safe_clip(p.Omega*ri*(1.0 + ap[i]) + Vt_ext, -300.0, 300.0)
            Un = safe_clip(p.Vax + a[i]*p.Omega*p.R, -150.0, 150.0)
            phi = np.arctan2(Un, Ut)
            q  = 0.5*p.rho*(safe_sq(Ut, 400.0) + safe_sq(Un, 200.0))
            L, D, _, _ = section_lift_drag(theta_loc - phi - AERO_MR.alpha0, ci, q, AERO_MR)
            Fcorr = prandtl_F_guarded(p.Nb, ri, p.R, p.Rhub, phi) if p.use_prandtl else 1.0
            Ft = (L*np.sin(phi) + D*np.cos(phi)) * dr[i] * Fcorr
            Fn = (L*np.cos(phi) - D*np.sin(phi)) * dr[i] * Fcorr
            Ft_x = Ft * (-np.sin(ps)); Ft_y = Ft * (np.cos(ps))
            rx = ri*np.cos(ps); ry = ri*np.sin(ps)
            Mx_ps += ry*Fn; My_ps += -rx*Fn; Q_ps += Ft*ri
            Fx_ps += Ft_x; Fy_ps += Ft_y; Fz_ps += Fn
        Fx += 2*Fx_ps; Fy += 2*Fy_ps; Fz += 2*Fz_ps
        Mx += 2*Mx_ps; My += 2*My_ps; Q  += 2*Q_ps
        hub_az.append([np.rad2deg(ps), 2*Fx_ps, 2*Fy_ps, 2*Fz_ps, 2*Mx_ps, 2*My_ps, 2*Q_ps])

    # power split (guarded)
    P_ind=P_prof=0.0
    for i, ri in enumerate(r):
        ci = chord[i]
        for ps in psi:
            Vt_ext = -p.Vx*np.sin(ps) + p.Vy*np.cos(ps)
            Ut = safe_clip(p.Omega*ri*(1.0 + ap[i]) + Vt_ext, -300.0, 300.0)
            Un = safe_clip(p.Vax + a[i]*p.Omega*p.R, -150.0, 150.0)
            phi = np.arctan2(Un, Ut)
            q = 0.5*p.rho*(safe_sq(Ut, 400.0) + safe_sq(Un, 200.0))
            L, D, _, _ = section_lift_drag(theta_geom[i] + elevon_delta_theta(p, ri, ps) - phi - AERO_MR.alpha0, ci, q, AERO_MR)
            Ft_ind = L*np.sin(phi); Ft_prof = D*np.cos(phi)
            Vt_loc = p.Omega*ri + Vt_ext
            P_ind  += Ft_ind  * Vt_loc * ri * dr[i] / az_count
            P_prof += Ft_prof * Vt_loc * ri * dr[i] / az_count

    # internal loads and stress surrogates (reuse your section_properties)
    # recompute Nn, Nt if needed (already az-averaged above)
    Vz   = np.cumsum(Nn[::-1]*dr[::-1])[::-1]
    Mfl  = np.cumsum((Nn[::-1]*dr[::-1])*r[::-1])[::-1]
    Ttor = np.cumsum((Nt[::-1]*dr[::-1])*r[::-1])[::-1]
    sigma_max = np.zeros_like(r); tau_max = np.zeros_like(r)
    for i in range(len(r)):
        Ixx, Iyy, J = section_properties(chord[i], sec)
        y_max = 0.06*chord[i]
        sigma_max[i] = Mfl[i]*y_max/max(Ixx, 1e-9)
        tau_max[i]   = Ttor[i]/max(J, 1e-9)

    return {
        "r": r, "dr": dr, "chord": chord,
        "a": a, "ap": ap,
        "Nn_per_span": Nn, "Nt_per_span": Nt,
        "Vz": Vz, "M_flap": Mfl, "T_torsion": Ttor,
        "sigma_flap_max": sigma_max, "tau_torsion_max": tau_max,
        "dT_r": dT_r, "dQ_r": dQ_r,
        "Fx": Fx, "Fy": Fy, "Fz": Fz, "Mx": Mx, "My": My, "Q": Q,
        "P_total": Q*p.Omega, "P_ind": P_ind, "P_prof": P_prof,
        "hub_az": np.array(hub_az),
        "iter_logs": iter_logs
    }

# 5) Replace the trim Newton step with LM-regularized solver and bounds

# Loosen bounds and start “colder”
# In your
#solve_trim_gauss_newton,
# change bounds and initial guess:
#• Bounds
# bounds_lo = np.array([ -2.0,  np.deg2rad(-5.0), np.deg2rad(-5.0), 0.00, 0.00 ])
# bounds_hi = np.array([  14.0,  np.deg2rad( 5.0), np.deg2rad( 5.0), 2.00, 2.00 ])
#• Initial guess
# x_init = np.array([ max(p0.theta_collect_deg, 2.0), 0.0, 0.0, 0.05, 0.05 ], dtype=float)

def solve_trim_gauss_newton(residual_fun, x0, it_max=40, log_every=5):

def solve_trim_gauss_newton(residual_fun, x0, it_max=40, log_every=5):
    x = x0.copy()
    mu = 1e-1
    damper0 = 0.5

    bounds_lo = np.array([ -2.0,  np.deg2rad(-5.0), np.deg2rad(-5.0), 0.00, 0.00 ])
    bounds_hi = np.array([  14.0,  np.deg2rad( 5.0), np.deg2rad( 5.0), 2.00, 2.00 ])

    logs = []
    R, aux = residual_fun(x)

    for it in range(1, it_max+1):
        logs.append((it, *x, *R))
        if np.linalg.norm(R) < 1e-2:
            break

        # Finite-difference Jacobian
        J = np.zeros((len(R), len(x)))
        eps = np.array([0.20, 0.01, 0.01, 0.05, 0.05])
        for j in range(len(x)):
            xp = x.copy(); xm = x.copy()
            xp[j] = x[j] + eps[j]
            xm[j] = x[j] - eps[j]
            Rp, _ = residual_fun(xp)
            Rm, _ = residual_fun(xm)
            J[:, j] = (Rp - Rm) / (2*eps[j])

        # LM step: (J^T J + mu I) dx = -J^T R
        JTJ = J.T @ J
        g   = J.T @ R
        try:
            dx = -np.linalg.solve(JTJ + mu*np.eye(JTJ.shape[0]), g)
        except np.linalg.LinAlgError:
            diag = np.diag(JTJ).copy()
            diag[diag < 1e-8] = 1e-8
            dx = -g / (diag + mu)

        # Backtracking with bounds
        accepted = False
        damper = damper0
        for _ in range(7):
            x_try = np.minimum(np.maximum(x + damper*dx, bounds_lo), bounds_hi)
            R_try, _ = residual_fun(x_try)
            if np.isfinite(R_try).all() and np.linalg.norm(R_try) <= 1.05*np.linalg.norm(R):
                x = x_try; R = R_try
                mu = max(mu*0.7, 1e-6)
                accepted = True
                break
            damper *= 0.5
            mu = min(mu*2.0, 1e3)

        if not accepted:
            # small collective nudge if stuck
            x[0] = np.clip(x[0] + (-np.sign(R[0]))*0.2, bounds_lo[0], bounds_hi[0])

        if (it % log_every) == 0:
            logs.append((it, *x, *R))

    return x, R, logs, aux


# 6) Hook into your existing trim function:
# In your trim_main_with_props_and_exports, replace the Gauss-Newton loop with:
#
#   def residual(xvec):
#       ... build R and aux exactly as before, but ensure all math uses safe_clip, section_lift_drag, prandtl_F_guarded ...
#       return np.array([R1, R2, R3, R4]), aux
#
#   x_init = np.array([p0.theta_collect_deg, 0.0, 0.0, 1.0, 1.0], dtype=float)
#   x, R, trim_logs, aux = solve_trim_gauss_newton(residual, x_init, it_max=40, log_every=5)
#
# Keep writing the iteration CSV exactly as before using trim_logs.
#
# 7) Optional: start with slightly milder defaults for stability on first run
#    - MRParams().theta_collect_deg = 8.0
#    - MRParams().max_iter = 60
#    - MRParams().tol = 1e-3
#    - Reduce Nelem to 22 and az_count to 20 for the first validation; increase later.

In [ ]:

# Stabilized trim function using guarded BEMT and LM solver

def prop_section_and_azimuth_guarded(rho, prop: PropParams, theta_p_deg, omega, Nelem=None, az_count=24):
    # identical to your earlier prop_section_and_azimuth but with small guards
    if Nelem is None:
        Nelem = prop.Nelem
    r = np.linspace(0.15*prop.Rp, prop.Rp, Nelem)
    dr = np.gradient(r)
    chord = (prop.sigma_p * np.pi * prop.Rp) / prop.Bp
    chord_arr = np.full_like(r, chord)
    psi = np.linspace(0, 2*np.pi, az_count, endpoint=False)

    Nn = np.zeros_like(r); Nt = np.zeros_like(r)
    for i, ri in enumerate(r):
        UT = safe_clip(omega * ri, -400.0, 400.0)
        for _ in psi:
            phi = 0.0
            alpha = np.deg2rad(theta_p_deg) - phi - AERO_PROP.alpha0
            q = 0.5 * rho * safe_sq(UT, 400.0)
            L, D, _, _ = section_lift_drag(alpha, chord, q, AERO_PROP)
            Nn[i] += (L*np.cos(phi) - D*np.sin(phi))
            Nt[i] += (L*np.sin(phi) + D*np.cos(phi))
        Nn[i] /= len(psi); Nt[i] /= len(psi)

    T = np.sum(Nn*dr) * prop.Bp
    Q = np.sum(Nt*r*dr) * prop.Bp
    P = Q * omega

    Vref = omega * prop.Rp * 0.7
    qref = 0.5 * rho * safe_sq(Vref, 400.0)
    phi_mid = 0.0
    beta = (qref * AERO_PROP.Cla * (np.deg2rad(theta_p_deg) - phi_mid) * prop.e_beta) / (prop.kbeta + prop.m_eff*safe_sq(omega*prop.Rp, 400.0) + 1e-9)

    Ft_tot = np.sum(Nt*dr) * prop.Bp
    Fn_tot = np.sum(Nn*dr) * prop.Bp
    az = []
    for ps in psi:
        Fx_ps = Fn_tot * np.sin(beta)
        Fy_ps = 0.0
        Fz_ps = Fn_tot * np.cos(beta)
        Mx_ps = 0.0; My_ps = 0.0
        Q_ps  = Ft_tot * (prop.Rp*2/3.0)
        az.append([np.rad2deg(ps), Fx_ps, Fy_ps, Fz_ps, Mx_ps, My_ps, Q_ps])

    return {"r": r, "dr": dr, "chord": chord_arr,
            "Nn_per_span": Nn, "Nt_per_span": Nt,
            "T": T, "Q": Q, "P": P, "beta": beta,
            "hub_az": np.array(az)}

def prop_rpm_from_scale(omega_nom, S):
    S = float(np.maximum(S, 0.0))
    omega = omega_nom * np.sqrt(S)
    rpm = omega * 60.0/(2*np.pi)
    return omega, rpm

def pitt_peters_rhs(T, rho, R, A, Vx, Vy, Omega, lam):
    Vtip = abs(Omega * R) + 1e-6
    tau0 = R / (2*Vtip)
    tau1 = 1.5 * tau0
    A_pp = np.diag([-1.0/tau0, -1.0/tau1, -1.0/tau1])
    B_pp = np.array([
        [1.0/(2*rho*A*Vtip**2 + 1e-9), 0.0, 0.0],
        [0.0, 0.1/Vtip, 0.0],
        [0.0, 0.0, 0.1/Vtip]
    ])
    u = np.array([T, Vx, Vy])
    return A_pp @ lam + B_pp @ u, A_pp, B_pp

def trim_main_with_props_and_exports(mr: MRParams, prop: PropParams, sec_mr: SectionStruct,
                                     W_N, V_airspeed_mps,
                                     theta_p_in_deg, theta_p_out_deg,
                                     outdir, case_tag,
                                     export_sections=True, export_azimuth=True,
                                     log_every=5,
                                     az_count_mr=24, az_count_prop=24):
    make_dir(outdir)
    p0 = MRParams(**{**mr.__dict__})
    p0.Vx = float(V_airspeed_mps); p0.Vy = 0.0; p0.Vax = 0.0

    def residual(xvec):
        theta0, d1c, d1s, S_in, S_out = [float(v) for v in xvec]
        p = MRParams(**{**p0.__dict__})
        p.theta_collect_deg = theta0
        p.elevon_delta1c_deg = np.rad2deg(d1c)
        p.elevon_delta1s_deg = np.rad2deg(d1s)

        res_mr = mr_bemt_azimuth(p, sec_mr, az_count=az_count_mr, log_every=log_every)

        omega_in,  rpm_in  = prop_rpm_from_scale(prop.omega_in_nom,  S_in)
        omega_out, rpm_out = prop_rpm_from_scale(prop.omega_out_nom, S_out)

        res_pin  = prop_section_and_azimuth_guarded(p.rho, prop, theta_p_in_deg,  omega_in,  Nelem=prop.Nelem, az_count=az_count_prop)
        res_pout = prop_section_and_azimuth_guarded(p.rho, prop, theta_p_out_deg, omega_out, Nelem=prop.Nelem, az_count=az_count_prop)

        Tin  = res_pin["T"];  Tout  = res_pout["T"]
        betain  = res_pin["beta"]; betaout = res_pout["beta"]

        Fz_props = 2*(Tin*np.cos(betain)) + 2*(Tout*np.cos(betaout))
        Fx_props = 2*(Tin*np.sin(betain)) + 2*(Tout*np.sin(betaout))
        Q_drive  = 2*(res_pin["Q"] + prop.r_in*Tin) + 2*(res_pout["Q"] + prop.r_out*Tout)

        Mx_prop = 2*(prop.r_in*Tin*np.cos(betain)) + 2*(prop.r_out*Tout*np.cos(betaout))
        My_prop = 0.0

        R1 = (res_mr["Fz"] + Fz_props) - W_N
        R2 = (res_mr["Mx"] + Mx_prop)
        R3 = (res_mr["My"] + My_prop)
        R4 = (res_mr["Q"] + Q_drive)

        aux = dict(p=p, rpm_in=rpm_in, rpm_out=rpm_out,
                   res_mr=res_mr, res_pin=res_pin, res_pout=res_pout,
                   Tin=Tin, Tout=Tout, betain=betain, betaout=betaout,
                   Fz_props=Fz_props, Fx_props=Fx_props, Q_drive=Q_drive)
        R = np.array([R1, R2, R3, R4], dtype=float)
        # Scale the residuals inside residual(xvec)
        # Right before “return R, aux”, normalize the residuals to
        # improve conditioning:
        # • Choose scales
        W = W_N
        MR = W_N * mr.R
        QR = W_N * mr.R
        QT = max(abs(res_mr["Q"]) + 1e5, 1e5)
        # • Apply
        R = np.array([ R1/W, R2/MR, R3/MR, R4/QT ], dtype=float)

        # residual scaling for conditioning
        W_scale  = W_N M_scale  = max(W_N * mr.R, 1.0)
        Q_scale  = max(abs(res_mr["Q"]) + 1e5, 1e5)
        R_unscaled = np.array([R1, R2, R3, R4], dtype=float)
        R = np.array([R1 / W_scale, R2 / M_scale, R3 / M_scale, R4 / Q_scale], dtype=float)
        aux.update({ "R_unscaled": R_unscaled,
                     "scales": {"W": W_scale, "M": M_scale, "Q": Q_scale} })


        return R, aux

    # Solve with LM and bounds
    # x_init = np.array([p0.theta_collect_deg, 0.0, 0.0, 1.0, 1.0], dtype=float)
    # x_init = np.array([ max(p0.theta_collect_deg, 2.0), 0.0, 0.0, 0.05, 0.05 ], dtype=float)
    x_init = np.array([max(p0.theta_collect_deg, 2.0), 0.0, 0.0, 0.05, 0.05], dtype=float)

    t0 = time.time()
    x, R, trim_logs, aux = solve_trim_gauss_newton(residual, x_init, it_max=40, log_every=log_every)
    elapsed = time.time() - t0

    theta0, d1c, d1s, S_in, S_out = x
    res_mr   = aux["res_mr"]; res_pin = aux["res_pin"]; res_pout = aux["res_pout"]
    rpm_in   = aux["rpm_in"]; rpm_out = aux["rpm_out"]

    P_main = res_mr["P_total"]; P_ind = res_mr["P_ind"]; P_prof = res_mr["P_prof"]
    P_props = 2*res_pin["P"] + 2*res_pout["P"]
    P_equiv = P_main + P_props


# -----------------------
    # EXPORTS AND EIGENVALUES
    # -----------------------
    case_base = os.path.join(outdir, case_tag)
    os.makedirs(outdir, exist_ok=True)

    # Iteration logs (trim + inflow)
    it_csv = case_base + "_iterations.csv"
    with open(it_csv, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["phase","iter","theta0_deg","d1c_rad","d1s_rad","S_in","S_out",
                    "R1","R2","R3","R4","norm_a","norm_ap","Fz_est","Q_est","elapsed_s"])
        # Trim iterations (scaled residuals are stored in trim_logs)
        last_it = trim_logs[-1][0] if len

    # Export iterations
    case_base = os.path.join(outdir, case_tag)
    it_csv = case_base + "_iterations.csv"
    with open(it_csv, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["phase","iter","theta0_deg","d1c_rad","d1s_rad","S_in","S_out",
                    "R1","R2","R3","R4","norm_a","norm_ap","Fz_est","Q_est","elapsed_s"])
        last_it = trim_logs[-1][0] if len(trim_logs) else 0
        for (it, th, d1c_v, d1s_v, Sin, Sout, R1,R2,R3,R4) in trim_logs:
            if (it % log_every == 0) or (it == last_it):
                w.writerow(["trim", it, th, d1c_v, d1s_v, Sin, Sout, R1,R2,R3,R4, "", "", "", "", elapsed])
        for (it, na, nap, Fz_est, Q_est) in res_mr["iter_logs"]:
            w.writerow(["inflow", it, "", "", "", "", "", "", "", "", "", na, nap, Fz_est, Q_est, elapsed])

    # Export MR sections
    with open(case_base + "_MR_sections.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["r_m","dr_m","chord_m","a_axial","a_swirl",
                    "Nn_per_span_Npm","Nt_per_span_Npm",
                    "Vz_N","M_flap_Nm","T_torsion_Nm",
                    "sigma_flap_max_Pa","tau_torsion_max_Pa",
                    "dT_annulus_N","dQ_annulus_Nm"])
        for i in range(len(res_mr["r"])):
            w.writerow([res_mr["r"][i], res_mr["dr"][i], res_mr["chord"][i],
                        res_mr["a"][i], res_mr["ap"][i],
                        res_mr["Nn_per_span"][i], res_mr["Nt_per_span"][i],
                        res_mr["Vz"][i], res_mr["M_flap"][i], res_mr["T_torsion"][i],
                        res_mr["sigma_flap_max"][i], res_mr["tau_torsion_max"][i],
                        res_mr["dT_r"][i], res_mr["dQ_r"][i]])

    # Export MR azimuth
    with open(case_base + "_MR_azimuth.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["psi_deg","Fx_N","Fy_N","Fz_N","Mx_Nm","My_Nm","Q_Nm"])
        for row in res_mr["hub_az"]:
            w.writerow(list(row))

    # Props sections and azimuth
    with open(case_base + "_PropIN_sections.csv", "w", newline="") as f:
        w = csv.writer(f); w.writerow(["r_m","dr_m","chord_m","Nn_per_span_Npm","Nt_per_span_Npm"])
        for i in range(len(res_pin["r"])):
            w.writerow([res_pin["r"][i], res_pin["dr"][i], res_pin["chord"][i],
                        res_pin["Nn_per_span"][i], res_pin["Nt_per_span"][i]])
    with open(case_base + "_PropOUT_sections.csv", "w", newline="") as f:
        w = csv.writer(f); w.writerow(["r_m","dr_m","chord_m","Nn_per_span_Npm","Nt_per_span_Npm"])
        for i in range(len(res_pout["r"])):
            w.writerow([res_pout["r"][i], res_pout["dr"][i], res_pout["chord"][i],
                        res_pout["Nn_per_span"][i], res_pout["Nt_per_span"][i]])
    with open(case_base + "_PropIN_azimuth.csv", "w", newline="") as f:
        w = csv.writer(f); w.writerow(["psi_deg","Fx_N","Fy_N","Fz_N","Mx_Nm","My_Nm","Q_Nm"])
        for row in res_pin["hub_az"]:
            w.writerow(list(row))
    with open(case_base + "_PropOUT_azimuth.csv", "w", newline="") as f:
        w = csv.writer(f); w.writerow(["psi_deg","Fx_N","Fy_N","Fz_N","Mx_Nm","My_Nm","Q_Nm"])
        for row in res_pout["hub_az"]:
            w.writerow(list(row))

    # Dynamic inflow eigenvalues and controllability (Pitt–Peters)
    lam0 = np.zeros(3)
    Aref = np.pi*p0.R**2
    _, A_pp_mr, B_pp_mr = pitt_peters_rhs(res_mr["Fz"], p0.rho, p0.R, Aref, p0.Vx, p0.Vy, p0.Omega, lam0)
    eig_pp = np.linalg.eigvals(A_pp_mr)
    C_pp = np.concatenate([B_pp_mr, A_pp_mr@B_pp_mr, A_pp_mr@A_pp_mr@B_pp_mr], axis=1)
    rank_pp = np.linalg.matrix_rank(C_pp)

    # Reduced 2-state surrogate (as before; diagonal stable A)
    eig_red = np.array([-1.0, -1.0], dtype=float)
    rank_red = 0

    # Per-case summary CSV
    case_csv = case_base + "_case.csv"
    with open(case_csv, "w", newline="") as f:
        w = csv.writer(f); w.writerow(["key","value"])
        w.writerow(["theta0_deg", theta0])
        w.writerow(["elevon1c_deg", np.rad2deg(d1c)])
        w.writerow(["elevon1s_deg", np.rad2deg(d1s)])
        w.writerow(["rpm_in", rpm_in])
        w.writerow(["rpm_out", rpm_out])
        w.writerow(["Fz_main_N", res_mr["Fz"]])
        w.writerow(["Fz_props_N", aux["Fz_props"]])
        w.writerow(["Fx_props_N", aux["Fx_props"]])
        w.writerow(["Mx_main_Nm", res_mr["Mx"]])
        w.writerow(["My_main_Nm", res_mr["My"]])
        w.writerow(["Q_aero_Nm", res_mr["Q"]])
        w.writerow(["Q_drive_Nm", aux["Q_drive"]])
        w.writerow(["P_main_W", P_main])
        w.writerow(["P_ind_W", P_ind])
        w.writerow(["P_prof_W", P_prof])
        w.writerow(["P_props_W", P_props])
        w.writerow(["P_equiv_W", P_equiv])
        w.writerow(["eig_pp_r1", float(np.real(eig_pp[0]))])
        w.writerow(["eig_pp_i1", float(np.imag(eig_pp[0]))])
        w.writerow(["eig_pp_r2", float(np.real(eig_pp[1]))])
        w.writerow(["eig_pp_i2", float(np.imag(eig_pp[1]))])
        w.writerow(["eig_pp_r3", float(np.real(eig_pp[2]))])
        w.writerow(["eig_pp_i3", float(np.imag(eig_pp[2]))])
        w.writerow(["ctrb_rank_pp", int(rank_pp)])
        w.writerow(["eig_red_r1", float(np.real(eig_red[0]))])
        w.writerow(["eig_red_i1", float(np.imag(eig_red[0]))])
        w.writerow(["eig_red_r2", float(np.real(eig_red[1]))])
        w.writerow(["eig_red_i2", float(np.imag(eig_red[1]))])
        w.writerow(["ctrb_rank_red", int(rank_red)])
        w.writerow(["trim_residual_norm", float(np.linalg.norm(R))])
        w.writerow(["elapsed_s", elapsed])

    return dict(theta0_deg=theta0, elevon1c_deg=np.rad2deg(d1c), elevon1s_deg=np.rad2deg(d1s),
                rpm_in=rpm_in, rpm_out=rpm_out,
                Fz_main=res_mr["Fz"], Fz_props=aux["Fz_props"], Fx_props=aux["Fx_props"],
                Mx_main=res_mr["Mx"], My_main=res_mr["My"], Q_aero=res_mr["Q"], Q_drive=aux["Q_drive"],
                P_main=P_main, P_ind=P_ind, P_prof=P_prof, P_props=P_props, P_equiv=P_equiv,
                eig_pp=eig_pp, ctrb_rank_pp=rank_pp,
                eig_red=eig_red, ctrb_rank_red=rank_red,
                residual_norm=float(np.linalg.norm(R)), elapsed_s=elapsed)

In [ ]:

# Reduce fidelity for first run if needed
mr = MRParams(Nelem=22, max_iter=60, tol=1e-3)
prop = PropParams(Nelem=14)
sec = SectionStruct()

outdir = "cases_output"
case = "W39000lb_V0kt_stable_smoke"

res = trim_main_with_props_and_exports(
    mr, prop, sec,
    W_N = 39000.0 * LBF2N,
    V_airspeed_mps = 0.0 * KT2MPS,
    theta_p_in_deg = prop.theta_p_in_deg,
    theta_p_out_deg= prop.theta_p_out_deg,
    outdir = outdir,
    case_tag = case,
    export_sections = True,
    export_azimuth = True,
    log_every = 5,
    az_count_mr = 20,
    az_count_prop = 20
)
print("Smoke test done. Residual norm =", res["residual_norm"])

KeyboardInterrupt: 

In [ ]:
# Cell 4: Propellers + dynamic inflow

def prop_section_and_azimuth(rho, prop: PropParams, theta_p_deg, omega, Nelem=None, az_count=24):
    if Nelem is None:
        Nelem = prop.Nelem
    r = np.linspace(0.15*prop.Rp, prop.Rp, Nelem)
    dr = np.gradient(r)
    chord = (prop.sigma_p * np.pi * prop.Rp) / prop.Bp
    chord_arr = np.full_like(r, chord)
    psi = np.linspace(0, 2*np.pi, az_count, endpoint=False)

    Nn = np.zeros_like(r); Nt = np.zeros_like(r)
    for i, ri in enumerate(r):
        UT = omega * ri
        for _ in psi:
            phi = 0.0
            alpha = np.deg2rad(theta_p_deg) - phi - prop.alpha0
            Cl = prop.Cla * alpha
            Cd = prop.CD0 + prop.kCD*Cl*Cl
            q = 0.5 * rho * (UT**2)
            L = q * chord * Cl
            D = q * chord * Cd
            Nn[i] += (L*np.cos(phi) - D*np.sin(phi))
            Nt[i] += (L*np.sin(phi) + D*np.cos(phi))
        Nn[i] /= len(psi); Nt[i] /= len(psi)

    T = np.sum(Nn*dr) * prop.Bp
    Q = np.sum(Nt*r*dr) * prop.Bp
    P = Q * omega

    # Flapping small-angle
    Vref = omega * prop.Rp * 0.7
    qref = 0.5 * rho * Vref*Vref
    phi_mid = 0.0
    beta = (qref * prop.Cla * (np.deg2rad(theta_p_deg) - phi_mid) * prop.e_beta) / (prop.kbeta + prop.m_eff*(omega*prop.Rp)**2 + 1e-9)

    Ft_tot = np.sum(Nt*dr) * prop.Bp
    Fn_tot = np.sum(Nn*dr) * prop.Bp
    az = []
    for ps in psi:
        Fx_ps = Fn_tot * np.sin(beta)
        Fy_ps = 0.0
        Fz_ps = Fn_tot * np.cos(beta)
        Mx_ps = 0.0; My_ps = 0.0
        Q_ps  = Ft_tot * (prop.Rp*2/3.0)
        az.append([np.rad2deg(ps), Fx_ps, Fy_ps, Fz_ps, Mx_ps, My_ps, Q_ps])

    return {
        "r": r, "dr": dr, "chord": chord_arr,
        "Nn_per_span": Nn, "Nt_per_span": Nt,
        "T": T, "Q": Q, "P": P, "beta": beta,
        "hub_az": np.array(az)
    }

def prop_rpm_from_scale(omega_nom, S):
    return (omega_nom * np.sqrt(max(S,0.0))), (omega_nom * np.sqrt(max(S,0.0))) * 60.0/(2*np.pi)

def pitt_peters_rhs(T, rho, R, A, Vx, Vy, Omega, lam):
    Vtip = Omega * R + 1e-6
    tau0 = R / (2*Vtip)
    tau1 = 1.5 * tau0
    A_pp = np.diag([-1.0/tau0, -1.0/tau1, -1.0/tau1])
    B_pp = np.array([
        [1.0/(2*rho*A*Vtip**2 + 1e-9), 0.0, 0.0],
        [0.0, 0.1/Vtip, 0.0],
        [0.0, 0.0, 0.1/Vtip]
    ])
    u = np.array([T, Vx, Vy])
    return A_pp @ lam + B_pp @ u, A_pp, B_pp

Corrective actions (precise edits) Make these three small changes and re-run the smoke test.

1. Loosen bounds and start “colder” In your solve_trim_gauss_newton, change bounds and initial guess: • Bounds bounds_lo = np.array([ -2.0,  np.deg2rad(-5.0), np.deg2rad(-5.0), 0.00, 0.00 ]) bounds_hi = np.array([  14.0,  np.deg2rad( 5.0), np.deg2rad( 5.0), 2.00, 2.00 ]) • Initial guess x_init = np.array([ max(p0.theta_collect_deg, 2.0), 0.0, 0.0, 0.05, 0.05 ], dtype=float)


2. Scale the residuals inside residual(xvec) Right before “return R, aux”, normalize the residuals to improve conditioning: • Choose scales W = W_N MR = W_N * mr.R QR = W_N * mr.R QT = max(abs(res_mr["Q"]) + 1e5, 1e5) • Apply R = np.array([ R1/W, R2/MR, R3/MR, R4/QT ], dtype=float)


3. Slightly stronger regularization at start In solve_trim_gauss_newton set: • mu = 1e-1 • damper = 0.5

In [ ]:
# Cell 5: Trim solver + exporters

def trim_main_with_props_and_exports(mr: MRParams, prop: PropParams, sec_mr: SectionStruct,
                                     W_N, V_airspeed_mps,
                                     theta_p_in_deg, theta_p_out_deg,
                                     outdir, case_tag,
                                     export_sections=True, export_azimuth=True,
                                     log_every=5):
    make_dir(outdir)
    p0 = MRParams(**{**mr.__dict__})
    p0.Vx = V_airspeed_mps; p0.Vy = 0.0; p0.Vax = 0.0

    # Unknowns: [theta0, d1c, d1s, S_in, S_out]
    x = np.array([p0.theta_collect_deg, 0.0, 0.0, 1.0, 1.0], dtype=float)
    trim_logs = []  # (iter, theta0, d1c, d1s, S_in, S_out, R1,R2,R3,R4)

    def residual(xvec):
        theta0, d1c, d1s, S_in, S_out = xvec
        p = MRParams(**{**p0.__dict__})
        p.theta_collect_deg = theta0
        p.elevon_delta1c_deg = np.rad2deg(d1c)
        p.elevon_delta1s_deg = np.rad2deg(d1s)

        res_mr = mr_bemt_azimuth(p, sec_mr, az_count=24, log_every=log_every)

        omega_in, rpm_in   = prop_rpm_from_scale(prop.omega_in_nom,  S_in)
        omega_out, rpm_out = prop_rpm_from_scale(prop.omega_out_nom, S_out)

        res_pin  = prop_section_and_azimuth(p.rho, prop, theta_p_in_deg,  omega_in,  Nelem=prop.Nelem, az_count=24)
        res_pout = prop_section_and_azimuth(p.rho, prop, theta_p_out_deg, omega_out, Nelem=prop.Nelem, az_count=24)

        Tin = res_pin["T"];  Tout = res_pout["T"]
        betain  = res_pin["beta"]; betaout = res_pout["beta"]

        Fz_props = 2*(Tin*np.cos(betain)) + 2*(Tout*np.cos(betaout))
        Fx_props = 2*(Tin*np.sin(betain)) + 2*(Tout*np.sin(betaout))
        Q_drive  = 2*(res_pin["Q"] + prop.r_in*Tin) + 2*(res_pout["Q"] + prop.r_out*Tout)

        Mx_prop = 2*(prop.r_in*Tin*np.cos(betain)) + 2*(prop.r_out*Tout*np.cos(betaout))
        My_prop = 0.0

        R1 = (res_mr["Fz"] + Fz_props) - W_N
        R2 = (res_mr["Mx"] + Mx_prop)
        R3 = (res_mr["My"] + My_prop)
        R4 = (res_mr["Q"] + Q_drive)

        aux = dict(p=p, rpm_in=rpm_in, rpm_out=rpm_out,
                   res_mr=res_mr, res_pin=res_pin, res_pout=res_pout,
                   Tin=Tin, Tout=Tout, betain=betain, betaout=betaout,
                   Fz_props=Fz_props, Fx_props=Fx_props, Q_drive=Q_drive)
        return np.array([R1,R2,R3,R4]), aux

    t_case0 = time.time()
    for it in range(1, 40):
        R, aux = residual(x)
        trim_logs.append((it, *x, *R))
        if np.linalg.norm(R) < 1e-2:
            break
        J = np.zeros((4,5))
        eps = np.array([0.25, 0.01, 0.01, 0.05, 0.05])
        for j in range(5):
            xp = x.copy(); xp[j] += eps[j]
            Rp, _ = residual(xp)
            xm = x.copy(); xm[j] -= eps[j]
            Rm, _ = residual(xm)
            J[:,j] = (Rp - Rm) / (2*eps[j])
        dx, *_ = np.linalg.lstsq(J, -R, rcond=None)
        x += 0.7*dx
    elapsed = time.time() - t_case0

    R, aux = residual(x)
    theta0, d1c, d1s, S_in, S_out = x
    rpm_in  = aux["rpm_in"]; rpm_out = aux["rpm_out"]
    res_mr = aux["res_mr"]; res_pin = aux["res_pin"]; res_pout = aux["res_pout"]

    P_main = res_mr["P_total"]; P_ind = res_mr["P_ind"]; P_prof = res_mr["P_prof"]
    P_props = 2*res_pin["P"] + 2*res_pout["P"]
    P_equiv = P_main + P_props

    # Export iterations (every 5th and final for both trim and inflow)
    case_tag = f"{case_tag}"
    case_iter_csv = os.path.join(outdir, f"{case_tag}_iterations.csv")
    with open(case_iter_csv, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["phase","iter","theta0_deg","d1c_rad","d1s_rad","S_in","S_out",
                    "R1","R2","R3","R4","norm_a","norm_ap","Fz_est","Q_est","elapsed_s"])
        # trim logs
        last_it = trim_logs[-1][0]
        for (it, th, d1c_v, d1s_v, Sin, Sout, R1,R2,R3,R4) in trim_logs:
            if (it % 5 == 0) or (it == last_it):
                w.writerow(["trim", it, th, d1c_v, d1s_v, Sin, Sout, R1,R2,R3,R4, "", "", "", "", elapsed])
        # inflow logs
        for (it, na, nap, Fz_est, Q_est) in res_mr["iter_logs"]:
            w.writerow(["inflow", it, "", "", "", "", "", "", "", "", "", na, nap, Fz_est, Q_est, elapsed])

    # Export MR sections
    base = os.path.join(outdir, case_tag)
    with open(base + "_MR_sections.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["r_m","dr_m","chord_m","a_axial","a_swirl",
                    "Nn_per_span_Npm","Nt_per_span_Npm",
                    "Vz_N","M_flap_Nm","T_torsion_Nm",
                    "sigma_flap_max_Pa","tau_torsion_max_Pa",
                    "dT_annulus_N","dQ_annulus_Nm"])
        for i in range(len(res_mr["r"])):
            w.writerow([res_mr["r"][i], res_mr["dr"][i], res_mr["chord"][i],
                        res_mr["a"][i], res_mr["ap"][i],
                        res_mr["Nn_per_span"][i], res_mr["Nt_per_span"][i],
                        res_mr["Vz"][i], res_mr["M_flap"][i], res_mr["T_torsion"][i],
                        res_mr["sigma_flap_max"][i], res_mr["tau_torsion_max"][i],
                        res_mr["dT_r"][i], res_mr["dQ_r"][i]])

    # Export MR azimuth
    with open(base + "_MR_azimuth.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["psi_deg","Fx_N","Fy_N","Fz_N","Mx_Nm","My_Nm","Q_Nm"])
        for row in res_mr["hub_az"]:
            w.writerow(list(row))

    # Prop in/out sections
    with open(base + "_PropIN_sections.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["r_m","dr_m","chord_m","Nn_per_span_Npm","Nt_per_span_Npm"])
        for i in range(len(res_pin["r"])):
            w.writerow([res_pin["r"][i], res_pin["dr"][i], res_pin["chord"][i],
                        res_pin["Nn_per_span"][i], res_pin["Nt_per_span"][i]])
    with open(base + "_PropOUT_sections.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["r_m","dr_m","chord_m","Nn_per_span_Npm","Nt_per_span_Npm"])
        for i in range(len(res_pout["r"])):
            w.writerow([res_pout["r"][i], res_pout["dr"][i], res_pout["chord"][i],
                        res_pout["Nn_per_span"][i], res_pout["Nt_per_span"][i]])

    # Prop in/out azimuth
    with open(base + "_PropIN_azimuth.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["psi_deg","Fx_N","Fy_N","Fz_N","Mx_Nm","My_Nm","Q_Nm"])
        for row in res_pin["hub_az"]:
            w.writerow(list(row))
    with open(base + "_PropOUT_azimuth.csv", "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["psi_deg","Fx_N","Fy_N","Fz_N","Mx_Nm","My_Nm","Q_Nm"])
        for row in res_pout["hub_az"]:
            w.writerow(list(row))

    # Dynamic inflow eigenvalues/controllability (Pitt–Peters)
    lam0 = np.zeros(3)
    Aref = np.pi*p0.R**2
    _, A_pp_mr, B_pp_mr = pitt_peters_rhs(res_mr["Fz"], p0.rho, p0.R, Aref, p0.Vx, p0.Vy, p0.Omega, lam0)
    eig_mr = np.linalg.eigvals(A_pp_mr)
    C_mr = np.concatenate([B_pp_mr, A_pp_mr@B_pp_mr, A_pp_mr@A_pp_mr@B_pp_mr], axis=1)
    rank_mr = np.linalg.matrix_rank(C_mr)

    # Reduced inflow surrogate (2-state) for quick A,B,eigs
    def reduced_inflow_linearization(p_trim: MRParams):
        def outs(p):
            rr = mr_bemt_azimuth(p, sec_mr, az_count=18)
            return np.array([rr["Fz"], rr["Mx"], rr["My"]])
        y0 = outs(p_trim)
        eps = 1e-3
        Aout = np.zeros((3,2))
        pA = MRParams(**{**p_trim.__dict__}); pA.Vax += eps
        Aout[:,0] = (outs(pA)-y0)/eps
        pB = MRParams(**{**p_trim.__dict__}); pB.Vy += eps
        Aout[:,1] = (outs(pB)-y0)/eps
        kscale = max(1e-3, float(np.max(np.abs(Aout))))
        A_x = -kscale*np.eye(2)
        B_x = np.zeros((2,5))
        eig = np.linalg.eigvals(A_x)
        C = np.concatenate([B_x, A_x@B_x], axis=1) if B_x.size>0 else np.zeros((2,0))
        rank = np.linalg.matrix_rank(C) if B_x.size>0 else 0
        return A_x, B_x, eig, rank
    A_x_red, B_x_red, eig_red, rank_red = reduced_inflow_linearization(aux["p"])

    # Per-case summary CSV
    summary = dict(
        theta0_deg=theta0, elevon1c_deg=np.rad2deg(d1c), elevon1s_deg=np.rad2deg(d1s),
        rpm_in=rpm_in, rpm_out=rpm_out,
        Fz_main=res_mr["Fz"], Fz_props=aux["Fz_props"], Fx_props=aux["Fx_props"],
        Mx_main=res_mr["Mx"], My_main=res_mr["My"], Q_aero=res_mr["Q"], Q_drive=aux["Q_drive"],
        P_main=P_main, P_ind=P_ind, P_prof=P_prof, P_props=P_props, P_equiv=P_equiv,
        eig_pp_mr_real=list(np.real(eig_mr)), eig_pp_mr_imag=list(np.imag(eig_mr)), ctrb_rank_pp=int(rank_mr),
        eig_red_real=list(np.real(eig_red)), eig_red_imag=list(np.imag(eig_red)), ctrb_rank_red=int(rank_red),
        residual_norm=float(np.linalg.norm(R)), elapsed_s=elapsed
    )
    with open(base + "_case.csv", "w", newline="") as f:
        w = csv.writer(f); w.writerow(["key","value"])
        for k,v in summary.items():
            w.writerow([k, v])

    return summary

In [ ]:
# Cell 6: Batch run and packaging

def run_batch_and_package(output_dir="cases_output",
                          weights_lb=(39000.0, 109000.0),
                          speeds_kt=(0.0, 20.0, 40.0, 60.0),
                          export_sections=True, export_azimuth=True):
    mr = MRParams()
    prop = PropParams()
    sec_mr = SectionStruct()

    make_dir(output_dir)
    master_csv = os.path.join(output_dir, "all_cases.csv")
    header = ["weight_lb","V_kt","theta0_deg","elevon1c_deg","elevon1s_deg",
              "rpm_in","rpm_out",
              "Fz_main_N","Fz_props_N","Fx_props_N",
              "Mx_main_Nm","My_main_Nm","Q_aero_Nm","Q_drive_Nm",
              "P_main_W","P_ind_W","P_prof_W","P_props_W","P_equiv_W",
              "eig_pp_mr_r1","eig_pp_mr_i1","eig_pp_mr_r2","eig_pp_mr_i2","eig_pp_mr_r3","eig_pp_mr_i3",
              "ctrb_rank_pp",
              "eig_red_r1","eig_red_i1","eig_red_r2","eig_red_i2","ctrb_rank_red",
              "residual_norm","elapsed_s"]
    rows = []

    t0 = time.time()
    for Wlb in weights_lb:
        W = Wlb * LBF2N
        for Vkt in speeds_kt:
            V = Vkt * KT2MPS
            tag = f"W{int(Wlb)}lb_V{int(Vkt)}kt"
            print(f"Running case {tag} ...")
            summary = trim_main_with_props_and_exports(
                mr, prop, sec_mr, W, V,
                prop.theta_p_in_deg, prop.theta_p_out_deg,
                output_dir, tag,
                export_sections=export_sections, export_azimuth=export_azimuth, log_every=5
            )
            eig_pp = list(zip(summary["eig_pp_mr_real"], summary["eig_pp_mr_imag"]))
            while len(eig_pp) < 3:
                eig_pp.append((0.0,0.0))
            eig_red = list(zip(summary["eig_red_real"], summary["eig_red_imag"]))
            while len(eig_red) < 2:
                eig_red.append((0.0,0.0))

            row = [
                Wlb, Vkt,
                summary["theta0_deg"], summary["elevon1c_deg"], summary["elevon1s_deg"],
                summary["rpm_in"], summary["rpm_out"],
                summary["Fz_main"], summary["Fz_props"], summary["Fx_props"],
                summary["Mx_main"], summary["My_main"], summary["Q_aero"], summary["Q_drive"],
                summary["P_main"], summary["P_ind"], summary["P_prof"], summary["P_props"], summary["P_equiv"],
                eig_pp[0][0], eig_pp[0][1], eig_pp[1][0], eig_pp[1][1], eig_pp[2][0], eig_pp[2][1],
                summary["ctrb_rank_pp"],
                eig_red[0][0], eig_red[0][1], eig_red[1][0], eig_red[1][1], summary["ctrb_rank_red"],
                summary["residual_norm"], summary["elapsed_s"]
            ]
            rows.append(row)

    with open(master_csv, "w", newline="") as f:
        w = csv.writer(f); w.writerow(header); w.writerows(rows)

    zip_path = f"{output_dir}.zip"
    if os.path.exists(zip_path):
        os.remove(zip_path)
    shutil.make_archive(output_dir, "zip", output_dir)
    total_elapsed = time.time() - t0
    print(f"Master CSV: {master_csv}")
    print(f"Zipped results: {zip_path}")
    print(f"Total elapsed: {total_elapsed:.1f} s")
    return master_csv, zip_path

# To automatically run all cases when this cell is executed, uncomment:
# master_csv, zip_path = run_batch_and_package()
# from google.colab import files
# files.download(zip_path)

In [ ]:
# Cell 7: Run
# 1) Smoke test single case (fast)
_ = trim_main_with_props_and_exports(
    MRParams(), PropParams(), SectionStruct(),
    W_N = 39000.0 * LBF2N,
    V_airspeed_mps = 0.0 * KT2MPS,
    theta_p_in_deg = PropParams().theta_p_in_deg,
    theta_p_out_deg= PropParams().theta_p_out_deg,
    outdir = "cases_output",
    case_tag = "W39000lb_V0kt",
    export_sections = True,
    export_azimuth = True,
    log_every = 5
)
print("Smoke test completed. Per-case CSVs are in cases_output/")

# 2) Full batch across weights and speeds; enable to run the ~minutes job
master_csv, zip_path = run_batch_and_package(
    output_dir="cases_output",
    weights_lb=(39000.0, 109000.0),
    speeds_kt=(0.0, 20.0, 40.0, 60.0),
    export_sections=True,
    export_azimuth=True
)

# 3) Download the ZIP (uncomment in Colab runtime)
# from google.colab import files
# files.download(zip_path)